In [1]:
import requests
import codecs
import pickle
import numpy as np

In [2]:
import sys
sys.path.insert(0, '..')

In [3]:
from recommender_system_library.extra_functions.work_with_train_data import *

# Создание нового пользователя и получение токена

In [4]:
URL_NEW_USER = 'http://127.0.0.1:5000/registration'

In [5]:
result = requests.post(URL_NEW_USER)
token = result.json()['token'] if result.status_code < 300 else f'{result.status_code}: {result.json()}'
token

'Bm5aarABH6-J87hLBkZizg'

In [6]:
headers = {
    'token': token
}

# Создание новой модели

In [7]:
URL_CREATE = 'http://127.0.0.1:5000/create'

In [8]:
sgd_params_create = {
    'dimension': 30, 
    'learning_rate': 0.0001, 
    'user_regularization': 0.2, 
    'item_regularization': 0.3
}

In [9]:
model_type = 'latent_factor_sgd_model'

In [10]:
params_create = {
    'type': model_type,
    'params': sgd_params_create
}

In [11]:
result = requests.post(URL_CREATE, json=params_create, headers=headers)
model_id = result.json()['id'] if result.status_code < 300 else f'{result.status_code}: {result.json()}'
model_id

3

# Обучение модели

In [12]:
URL_TRAIN = 'http://127.0.0.1:5000/train/'

In [13]:
PATH_TO_DATA = '../data_train/matrices/random_matrix_50_100.npz'

In [14]:
matrix = read_matrix_from_file(PATH_TO_DATA)
train_matrix = get_train_matrix(matrix, 0.3)

In [15]:
embedding_train_params = {'epochs': 100}

In [16]:
params_train = {
    'params': embedding_train_params,
    'train_data': codecs.encode(pickle.dumps(train_matrix), 'base64').decode()
}

In [17]:
result = requests.post(URL_TRAIN + str(model_id), json=params_train, headers=headers)
result_print = 'OK' if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

'OK'

# Получение статуса модели

In [18]:
URL_STATUS = 'http://127.0.0.1:5000/status/'

In [19]:
result = requests.post(URL_STATUS + str(model_id), headers=headers)
result_print = result.json()['status'] if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

'TRAINING'

In [20]:
result = requests.post(URL_STATUS + str(model_id), headers=headers)
result_print = result.json()['status'] if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

'READY'

# Получение предсказания модели

In [21]:
URL_ITEMS = 'http://127.0.0.1:5000/predict_items/'
URL_RATINGS = 'http://127.0.0.1:5000/predict_ratings/'

In [22]:
params_predict = {
    'user': 13
}

In [23]:
result = requests.get(URL_ITEMS + str(model_id), json=params_predict, headers=headers)
result_print = result.json()['result'] if result.status_code < 300 else f'{result.status_code}: {result.json()}'
np.array(result_print)

array([96,  3, 81,  0, 42, 56, 39, 10, 70, 44, 92, 30, 66, 14, 93, 72, 57,
       38, 20, 77, 35, 51, 84, 47, 27,  8, 34, 37, 67, 28, 55, 97, 94, 23,
       43, 52,  5, 48, 19, 53, 65, 61, 91, 25, 26, 87, 36, 13, 89, 54, 24,
       63, 60, 74, 45, 32, 11,  4, 16, 62, 40,  9, 49, 85, 80, 90, 68, 59,
       71, 50,  7, 58, 88, 69, 83, 21, 15, 29,  6, 17, 22, 64, 78,  1, 33,
       41, 75, 76, 18, 12,  2, 31, 98, 79, 86, 73, 82, 99, 46, 95])

In [24]:
result = requests.get(URL_RATINGS + str(model_id), json=params_predict, headers=headers)
result_print = result.json()['result'] if result.status_code < 300 else f'{result.status_code}: {result.json()}'
np.array(result_print)

array([ 10.65456866,  -3.0669152 ,  -4.70604202,  12.05522334,
        -0.19415943,   2.37886417,  -2.69088175,  -2.02785985,
         4.00871021,  -1.04933479,   8.1958506 ,  -0.0909268 ,
        -4.59722041,   0.77521447,   5.95323295,  -2.47448412,
        -0.2801793 ,  -2.74303736,  -4.57142161,   2.09754901,
         4.91114298,  -2.3634933 ,  -2.78495335,   2.50498521,
         0.40234417,   1.24034984,   1.1743326 ,   4.05371916,
         3.58715425,  -2.51314244,   6.52407399,  -5.51116956,
        -0.03479903,  -4.10993686,   3.98254455,   4.38245163,
         1.05662154,   3.93421161,   5.10464557,   9.28457223,
        -0.99062616,  -4.15449933,  10.54600979,   2.49995432,
         7.40870641,   0.12403448, -10.34812906,   4.18677189,
         2.29897405,  -1.07545609,  -1.9047021 ,   4.3475257 ,
         2.49366939,   1.97283566,   0.57556008,   3.01433478,
         9.44380835,   5.24576316,  -2.08385976,  -1.79059981,
         0.1865839 ,   1.33371669,  -0.50701476,   0.35

# Дообучение модели

In [31]:
result = requests.post(URL_TRAIN + str(model_id), json=params_train, headers=headers)
result_print = 'OK' if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

'OK'

# Изменение модели

In [33]:
URL_CHANGE = 'http://127.0.0.1:5000/change/'

In [34]:
sgd_params_change = {
    'dimension': 35, 
    'learning_rate': 0.0001, 
    'user_regularization': 0.2, 
    'item_regularization': 0.3
}

In [37]:
params_not_change = {
    'params': sgd_params_create
}

In [38]:
params_change = {
    'params': sgd_params_change
}

In [39]:
result = requests.post(URL_CHANGE + str(model_id), json=params_not_change, headers=headers)
result_print = 'OK' if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

"405: {'message': 'The parameters are the same. Use clear handle'}"

In [41]:
result = requests.post(URL_CHANGE + str(model_id), json=params_change, headers=headers)
result_print = 'OK' if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

'OK'

# Очищение модели

In [45]:
URL_CLEAR = 'http://127.0.0.1:5000/clear/'

In [46]:
result = requests.post(URL_CLEAR + str(model_id), headers=headers)
result_print = 'OK' if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

'OK'

In [47]:
result = requests.get(URL_ITEMS + str(model_id), json=params_predict, headers=headers)
result_print = result.json()['result'] if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

"405: {'message': 'Model should be trained'}"

# Удаление модели

In [48]:
URL_DELETE = 'http://127.0.0.1:5000/delete/'

In [49]:
result = requests.delete(URL_DELETE + str(model_id), headers=headers)
result_print = 'OK' if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

'OK'

In [50]:
result = requests.get(URL_ITEMS + str(model_id), json=params_predict, headers=headers)
result_print = result.json()['result'] if result.status_code < 300 else f'{result.status_code}: {result.json()}'
result_print

"405: {'message': 'No access to this model'}"

# Неправильные запросы

In [51]:
result = requests.delete(URL_DELETE + str(model_id))
result.json()

{'message': 'Token is not valid'}

In [52]:
result = requests.delete(URL_DELETE + str(1), headers=headers)
result.json()

{'message': 'No access to this model'}

In [53]:
result = requests.post(URL_CREATE, json=params_change, headers=headers)
result.json()

{'message': 'No needed parameters'}